# Testing rotation of detector to change cake position in pyFAI

This notebook uses the test image found at: http://www.silx.org/pub/pyFAI/testimages/mock.tif
This is clearly an unrealistic scattering image but provides a good way to
show how different detector alignments affect the final data.

We look at a workflow for caking scattering spectra using pyFAI and then reading this data into `xrdfit`.

In [ ]:
import fabio
import xrdfit
from pyFAI.gui import jupyter
import pyFAI.detectors, pyFAI.azimuthalIntegrator
import math
import numpy as np
from xrdfit.spectrum_fitting import FitSpectrum

Load test image from file

In [ ]:
img = fabio.open(r"data/moke.tif").data
jupyter.display(img, label ="Fake diffraction image")

Set up mock detector *- normally you would load a calibration here.*
The parameters are those given with the sample image.

In [ ]:
detector = pyFAI.detectors.Detector(pixel1=1e-4, pixel2=1e-4)
ai = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=0.1, detector=detector)
# Center the detector on the center of the pattern at 300, 300.
ai.setFit2D(100, 300, 300)

Now rotate the detector. `rot3` is the angle to rotate the detector in the axis of the imaging beam.
See [here](https://pyfai.readthedocs.io/en/latest/usage/tutorial/Geometry/geometry.html#Conclusion) for a
schematic of the detector and rotations.

The minimum cake angle in pyFAI is -180${^\circ}$, and so caking will slice along the horizontal,
rather than being centred on the horizontal. We can centre the cake on the horizontal (and vertical)
by applying a rotation to the detector.

*Note, with 36 cakes and 100 slices the resolution is poor but xrdfit plotting is fast.
With 360 cakes and 1000 slices it is easier to see image reconstructed from the cakes but
xrdfit plotting is slow. This appears to be a limitation of the radial plotting in matplotlib.*

You can alter the number of cakes, radial slices and detector rotation here to see how it affects the
caked data.

In [ ]:
num_cakes = 36
num_radial_slices = 1000

# Rotate the detector so that the cardinal direction is in the center of the first cake.
cake_angle = 360 / num_cakes
ai.rot3 = (cake_angle / 2) * (math.pi / 180) # convert rotation to radians

Display the caked image in a flat 2D perspective.

In [ ]:
res2d = ai.integrate2d(img, num_radial_slices, num_cakes, unit="2th_deg")
jupyter.plot2d(res2d, label="moke")

pyFAI cakes the data anticlockwise (from -180${^\circ}$ to 0${^\circ}$ to +180${^\circ}$) which is the
opposite direction to DAWN. pyFAI also starts caking the data from the `West` (-180${^\circ}$) direction
(compared to DAWN which starts caking from the East direction) so the angle of the first cake is
270 degrees clockwise of the North direction.

To replicate a DAWN caked output the order of the cakes in the intensity variable need to be flipped.

By default, `np.savetxt` uses a space as a column delimiter. `xrdfit` defaults to tab spacing but
this can be changed using the `delimiter` argument.

We can pyFAI to reproduce the equivalent output of DAWN as:

In [ ]:
# Flip the intensity data to order cakes clockwise rather than anticlockwise
intensity = np.flip(res2d.intensity.T, axis=1)

# Reshape radial labels to 2D array so they can be attached to the intensity data.
radial = np.reshape(res2d.radial, (-1, 1))

result = np.hstack((radial, intensity))
np.savetxt("example_caked_spectrum.txt", result)
spectrum = xrdfit.spectrum_fitting.FitSpectrum("example_caked_spectrum.txt", first_cake_angle=270, delimiter=" ")
spectrum.plot_polar()

*Note, the color change between images is not important - it is that `xrdfit` is using a just a different matplotlib
colormap to the `jupyter.plot2d` function.*

An alternative to flipping the data is to use the new anticlockwise fit option of `xrdfit`.
This is particularly useful for users who have already gone through the process of caking their data using
pyFAI and would like to load it into xrdfit.

*Note, that if the data is read anticlockwise then the `first_cake_angle` has to be displaced by one cake width
as we are plotting data the other way.*

In [ ]:
# Just transpose rows/columns, dont flip.
intensity = res2d.intensity.T

# Reshape radial labels to 2D array so they can be attached to the intensity data.
radial = np.reshape(res2d.radial, (-1, 1))

result = np.hstack((radial, intensity))
np.savetxt("example_caked_spectrum.txt", result)
spectrum = xrdfit.spectrum_fitting.FitSpectrum("example_caked_spectrum.txt", first_cake_angle=270-cake_angle, delimiter=" ", cake_order="anticlockwise")
spectrum.plot_polar()